<a name="top"></a>Contents
===
- [Introduction](#intro)

- [Libraries](#libraries)

- [Datetime Dataframe](#datetime)

- [Pageviews](#pageviews)
    - [BW](#bw)
        - [Open BW dataset](#open_bw)
        - [Customize BW dataset](#customize_bw)
    - [Adwords](#adwords)
        - [Open Adwords dataset](#open_ads)
        - [Customize Ads dataset](#customize_ads)
    - [Merge BW and Adwords data](#merge_bw_ads)
    
- [References url](#url)
    - [Open References url dataset](#open_url)
    - [Merge References url and Top 50 references](#merge_url_top50)
    - [Customize References url dataset](#customize_url)

- [Looping through a dictionary](#looping)
    - [Looping through all key-value pairs](#loop_all_keys_values)
    - [Looping through all keys in a dictionary](#loop_all_keys)
    - [Looping through all values in a dictionary](#loop_all_values)
    - [Looping through a dictionary in order](#looping_in_order)
    - [Exercises](#exercises_looping)
    
- [Nesting](#nesting)
    - [Lists in a dictionary](#lists_in_dictionary)
    - [Dictionaries in a dictionary](#dictionaries_in_dictionary)
    - [An important note about nesting](#important_note)
    - [Exercises](#exercises_nesting)
    
- [Overall Challenges](#overall_challenges)

<a name='intro'></a>Introduction
===
EXPLICAR TODO Y PONER FOTO

<a name='libraries'></a>Libraries
===
First of all, it is necessary to import the required libraries to develop all the steps correctly of this notebbook.

In [129]:
import os
import io
import glob
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
%matplotlib inline

#seguramente haya que pones mas librerias

#display maximo columnas

#IMPORTAR WARNINGS

<a name='datetime'></a>  Datetime Dataframe
===
An inconvenient of the data selected for this study is missing values. 
For example, the advertising dataset has a missing value during a specific period when the reference has no impressions a day (it is possible to check an example in the photography atach)

### FOTO!!!!


For this reason, I am going to create a dataframe of a period of time and save it as csv with the purpose of merging with miss values from other datasets that I will work during this study.

In [12]:
path = '/home/miguel/my_project_python/my_project_env/tfm/shopping/notebook'
os.chdir(path)

In [13]:
%run -i 'date_dataframe.py'

<a name='pageviews'></a>Pageviews
===

In this notebook, I am going to merge all datasets link to pageviews information in order to create for each reference a csv file with the following data:
    - Reference
    - Date
    - Pageviews

In [ ]:
<a name='pageviews'></a>Pageviews
===

In this notebook, I am going to merge all datasets link to pageviews information in order to create for each reference a csv file with the following data:
    - Reference
    - Date
    - Pageviews

In [ ]:
First of all, I set the directory for the rest of notebook

In [68]:
path = '/home/miguel/my_project_python/my_project_env/tfm/shopping'
os.chdir(path)

In [ ]:
Then, I open the files required:

In [28]:
#select directory
path_reference_url = './data/customize/top_50'
path_datetime = './data/raw/datetime'


#load files
df_reference_url = pd.read_csv(path_reference_url+'/references_url.csv', sep='\t') 
df_datetime = pd.read_csv(path_datetime+'/datetime_df.csv')    #hacer funcion para datetime


#select columns
column_interest_reference = ['Reference', 'URL']
df_reference_url = df_reference_url[column_interest_reference]


column_interest_datetime = ['Dates']
df_datetime = df_datetime[column_interest_datetime]


#parse datetime
df_datetime['Dates'] = pd.to_datetime(df_datetime['Dates'], format = '%Y-%m-%d')

I check the data

In [29]:
df_reference_url.head(5)

,Reference,URL
0,122532,/espejo-pie-nordico-blanco-plastico-dormitorio...
1,51148,/consola-3-cajones-y-4-puertas-oriental-rojo-m...
2,51149,/consola-6-cajones-y-2-puertas-oriental-rojo-m...
3,51136,/consola-3-cajones-y-4-puertas-oriental-negro-...
4,51140,/mesilla-de-noche-1-cajon-y-2-puertas-oriental...


In [30]:
df_datetime.head(5)

,Dates
0,2018-09-03
1,2018-09-04
2,2018-09-05
3,2018-09-06
4,2018-09-07


After checking previous dataframe, I create a function to cover mainly the following steps:

- Open a csv file as dataframe
- Clean data from that file (rename columns, parse to datetime, etc.)
- Merge that dataframe with other dataframes
- Fill missing values
- Return the finaly dataframe

In [49]:
def pageview_missing_values(path,file):
    
    #os.chdir(path)
    
    df_pageview = pd.read_csv(path+file)
    
    df_pageview = df_pageview.rename(index=str, columns={'Day Index':'Dates', 'Page Views': 'Page_Views'})
    
    column_interest_pageview = ['Page', 'Dates', 'Page_Views']
    df_pageview = df_pageview[column_interest_pageview]   
    
    df_pageview['Dates'] = pd.to_datetime(df_pageview['Dates'], format = '%d/%m/%Y')
    
    merge_pageview = df_pageview.merge(df_datetime,
        how='right',
        left_on=['Dates'],
        right_on=['Dates'])
    
    merge_pageview['Page'].fillna(method='ffill', inplace = True)
    merge_pageview['Page_Views'].fillna(0, inplace = True)
    
    merge_url = merge_pageview.merge(df_reference_url,
        how='inner',
        left_on=['Page'],
        right_on=['URL'])
    
    column_interest_reference = ['Reference', 'Dates', 'Page_Views']
    merge_url = merge_url[column_interest_reference].sort_values(by='Dates')
        
    return merge_url

Now, I need to make a list to save the required data

In [50]:
lista = df_reference_url['Reference'].apply(str)+'_url.csv'
c = lista.values.tolist()

Finally, I make a for loop where I apply the funciont "pageview_missing_values" and save the dataframes as csv in a new directory

In [54]:
pwd

'/home/miguel/my_project_python/my_project_env/tfm/shopping'

In [56]:
for i in range(len(c)):
    
    path_raw_data = './data/raw/pageview/'
    
    df_test = pageview_missing_values(path_raw_data, c[i])
    file = c[i]
    
    path_save_data = './data/customize/pageviews/'
    #os.chdir(path)
    
    df_test.to_csv(path_save_data + file)

<a name='bw'></a>BW
===

In [67]:
def open_bw(file,sheet_name, path):

    #open the file xlsx    
    df_bw = pd.read_excel(path+file, sheet_name)
    
    #rename the file columns correctly
    df_bw = df_bw.rename(index=str, columns={'Referencia':'Reference', 'T Día natural':'Date', 
                                             'Valor neto pedidos':'Net_Incomes', 
                                             'Cantidad en unidades (pedidos)':'Units_sold'})
    
    #parse date columns
    df_bw['Date'] = pd.to_datetime(df_bw['Date'], format = '%d.%m.%Y')
    
    return df_bw

In [58]:
pwd

'/home/miguel/my_project_python/my_project_env/tfm/shopping'

In [69]:
#open the file
df_bw = open_bw('BW.xlsx', 'BW', './data/raw/bw/')

#head the file
df_bw.head(5)

,Reference,Name,Date,Net_Incomes,Units_sold
0,4623,"ESPEJO PUERTA ""LCC"" BLANCO POLIETILENO",2017-09-11,18.93,1
1,4623,"ESPEJO PUERTA ""LCC"" BLANCO POLIETILENO",2017-09-21,18.93,1
2,4623,"ESPEJO PUERTA ""LCC"" BLANCO POLIETILENO",2017-10-08,16.08,1
3,4623,"ESPEJO PUERTA ""LCC"" BLANCO POLIETILENO",2017-10-16,16.08,1
4,4623,"ESPEJO PUERTA ""LCC"" BLANCO POLIETILENO",2017-10-21,16.08,1


In [70]:
df_bw.info()

<class 'pandas.core.frame.DataFrame'>
Index: 37614 entries, 0 to 37613
Data columns (total 5 columns):
Reference      37614 non-null int64
Name           37614 non-null object
Date           37614 non-null datetime64[ns]
Net_Incomes    37614 non-null float64
Units_sold     37614 non-null int64
dtypes: datetime64[ns](1), float64(1), int64(2), object(1)
memory usage: 1.7+ MB


In [71]:
df_bw.shape

(37614, 5)

I also change "Reference" column from int64 to string in order to create a list later

In [95]:
df_bw['Reference'] = df_bw['Reference'].apply(str)

In [97]:
#create unique list of names
#UniqueNames = data.Names.unique()
referencess = df_bw['Reference'].unique().tolist()

#create a data frame dictionary to store your data frames
DataFrameDict = {elem : pd.DataFrame for elem in referencess}

for key in DataFrameDict.keys():
    DataFrameDict[key] = df_bw[:][df_bw.Reference == key]

I use a reference id as example to check if it is working correctly

In [98]:
DataFrameDict['4623']

,Reference,Name,Date,Net_Incomes,Units_sold
0,4623,"ESPEJO PUERTA ""LCC"" BLANCO POLIETILENO",2017-09-11,18.93,1
1,4623,"ESPEJO PUERTA ""LCC"" BLANCO POLIETILENO",2017-09-21,18.93,1
2,4623,"ESPEJO PUERTA ""LCC"" BLANCO POLIETILENO",2017-10-08,16.08,1
3,4623,"ESPEJO PUERTA ""LCC"" BLANCO POLIETILENO",2017-10-16,16.08,1
4,4623,"ESPEJO PUERTA ""LCC"" BLANCO POLIETILENO",2017-10-21,16.08,1


As we can see, it works well!

In [80]:
pwd

'/home/miguel/my_project_python/my_project_env/tfm/shopping'

In [87]:
#set directory
path = './data/customize/top_50/'

#open the file 'top_50'
df_top_50 = pd.read_csv(path+'references_url.csv', sep='\t')

#select columns
column_interest_reference = ['Reference']
df_top_50 = df_top_50[column_interest_reference]

In [88]:
df_top_50.head()

,Reference
0,122532
1,51148
2,51149
3,51136
4,51140


In [91]:
df_top_50.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 1 columns):
Reference    50 non-null int64
dtypes: int64(1)
memory usage: 480.0 bytes


I create another list using last dataset (I apply string to "Reference" column as I have also made before)

In [99]:
lista = df_top_50['Reference'].apply(str)
c = lista.values.tolist()

I make a for loop to:

- Read each component of the list and check its data from DataFrameDict.
- Set the directory to storage the data
- Save a csv file for each reference

In [101]:
for i in range(len(c)):
    
    df_test = DataFrameDict[c[i]]
    
    file = c[i]
    
    path = './data/customize/bw/'
        
    df_test.to_csv(path + file + '.csv')

Check the saved data

In [105]:
df_101318 = pd.read_csv('./data/customize/bw/101318.csv', sep=',', error_bad_lines=True)
df_101318.head()

,Unnamed: 0,Reference,Name,Date,Net_Incomes,Units_sold
0,21074,101318,SILLA MARRÓN PU/METAL SALÓN,2017-10-05,151.53,3
1,21075,101318,SILLA MARRÓN PU/METAL SALÓN,2017-10-12,118.85,2
2,21076,101318,SILLA MARRÓN PU/METAL SALÓN,2017-10-15,202.05,4
3,21077,101318,SILLA MARRÓN PU/METAL SALÓN,2018-02-06,44.55,1
4,21078,101318,SILLA MARRÓN PU/METAL SALÓN,2018-02-22,237.69,4


I create function "bw_missing_values" to be able to achieve following goals:

- Open some files that I will work
- Make the required data wrangling in this dataframes
- Work specificly in each reference
- Return a dataframe with the main data in each case

In [115]:
def bw_missing_values(path1, path2, file):
    
    #set directory and open datetime_df file
    #os.chdir(path1)
    df_datetime = pd.read_csv(path1 + 'datetime_df.csv') ###ABRIR CON LA FUNCION
    
    #selecto column interest and parse data to datetime
    column_interest_datetime = ['Dates']
    df_datetime = df_datetime[column_interest_datetime]
    df_datetime['Dates'] = pd.to_datetime(df_datetime['Dates'], format = '%Y-%m-%d')
    
        
    #set directory to open the reference file
    #os.chdir(path2)
    
    #open the csv file for each reference
    df_bw_ref = pd.read_csv(path2 + c[i] + '.csv')
        
    #data wranggling of this data        
    df_bw_ref['Date'] = pd.to_datetime(df_bw_ref['Date'], format = '%Y-%m-%d')
    df_bw_ref = df_bw_ref.rename(index=str, columns={'Net Incomes': 'Net_Incomes', })
        
    #select columns 
    column_interest_bw = ['Reference', 'Name', 'Date', 'Net_Incomes', 'Units_sold']
    df_bw_ref = df_bw_ref[column_interest_bw]
        
    #merge with df_datetime
    merge_bw = df_bw_ref.merge(df_datetime,
        how='right',
        left_on=['Date'],
        right_on=['Dates'])
        
    #fill NaN values
    merge_bw['Reference'].fillna(method='ffill', inplace = True)
    merge_bw['Name'].fillna(method='ffill', inplace = True)
    merge_bw.fillna(0, inplace = True)
    
    #sort values by Dates
    merge_bw = merge_bw.sort_values(by='Dates')
    
    #select required columns
    column_interest_merge = ['Reference', 'Name', 'Dates', 'Net_Incomes', 'Units_sold']
    merge_bw = merge_bw[column_interest_merge]   

        
    return merge_bw

I apply the previous function using a for loop

In [116]:
path_datetime = './data/raw/datetime/'
path_original_data = './data/customize/bw/'

for i in range(len(c)):
    df_test = bw_missing_values(path_datetime, path_original_data, c[i])
    file = c[i]
    
    path = './data/customize/merge_bw_pageviews/'
    #os.chdir(path)
    
    df_test.to_csv(path + file + '.csv')

In [ ]:
pwd

In [118]:
df_101318 = pd.read_csv('./data/customize/merge_bw_pageviews/101318.csv', sep=',', error_bad_lines=True)
df_101318.head()

,Unnamed: 0,Reference,Name,Dates,Net_Incomes,Units_sold
0,19,101318.0,SILLA MARRÓN PU/METAL SALÓN,2018-09-03,0.0,0.0
1,20,101318.0,SILLA MARRÓN PU/METAL SALÓN,2018-09-04,0.0,0.0
2,21,101318.0,SILLA MARRÓN PU/METAL SALÓN,2018-09-05,0.0,0.0
3,22,101318.0,SILLA MARRÓN PU/METAL SALÓN,2018-09-06,0.0,0.0
4,23,101318.0,SILLA MARRÓN PU/METAL SALÓN,2018-09-07,0.0,0.0


<a name='adwords'></a>Adwords
==

In this notebook, I am going to work with adwords data:
    - Load datasets.
    - Concatenate these datasets into one main dataset
    - Clean data (rename, select columns, etc.)
    - Select specific references and their data from previous dataset.
    - Split the main dataframe into multiple dataframes (a dataframe per reference)
    - Merge multiples dataframes with "datatime_df"
    - Save multiples dataframes correctly as csv files

I open adwords data. I have to open three files according to different time periods:
- TFM_Datos_2017_18_1.csv
- TFM_Datos_2017_18_2.csv
- TFM_Datos_2018_19.csv

In [120]:
pwd

'/home/miguel/my_project_python/my_project_env/tfm/shopping'

-------
QUITAR PRIMER FICHERO
SOLOS LOS DOS ULTIMOS Y FILTRAR
-------

In [130]:
#path = './data/raw/adwords/'

extension = 'csv'
all_filenames = [i for i in glob.glob('./data/raw/adwords/*.{}'.format(extension))]

#combine all files in the list
df_adwords = pd.concat([pd.read_csv(f, sep=";", error_bad_lines=True) for f in all_filenames])

/home/miguel/my_project_python/my_project_env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3214: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


In [131]:
df_adwords.head(5)

,ID de producto,Campaña,ID de la campaña,Tipo de producto (primer nivel),Tipo de producto (segundo nivel),Día,Etiqueta personalizada 1,CPC máximo predeterminado del grupo de anuncios,Moneda,Impresiones,Clics,CTR,CPC medio,Coste,Conversiones,Todas las conversiones,Valor de conv.,Valor de todas las conversiones,Valor conv./coste
0,123624,Shop_Cocina y comedor_N1_Y18_W37,1559515409,cocina y comedor,tazas de café y mugs,12/10/2018,10,"0,15",EUR,1,0,"0,00 %","0,00","0,00","0,00","0,00","0,00","0,00","0,00"
1,107553,Shop_Smart_Lámparas_Y18_W45,1623248621,lámparas e iluminación,lámparas de techo,31/01/2019,40,"0,01",EUR,2,0,"0,00 %","0,00","0,00","0,00","0,00","0,00","0,00","0,00"
2,87810,Shop_Navidad_Estrellas de navidad_Y18_W12,1323457572,decoración de navidad,estrellas de navidad,17/10/2018,20,"0,2",EUR,1,0,"0,00 %","0,00","0,00","0,00","0,00","0,00","0,00","0,00"
3,105114,Shop_Decoración_N1_Y18_W37,1559890739,decoración para tu casa,figuras decorativas,25/10/2018,30,"0,15",EUR,4,0,"0,00 %","0,00","0,00","0,00","0,00","0,00","0,00","0,00"
4,111017,Shop_Navidad_Portavelas navideños_Y18_W12,1323682318,decoración de navidad,portavelas navideños,21/10/2018,20,"0,2",EUR,8,0,"0,00 %","0,00","0,00","0,00","0,00","0,00","0,00","0,00"


In [132]:
df_adwords.shape

(1026615, 19)

In [133]:
df_adwords.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1026615 entries, 0 to 579075
Data columns (total 19 columns):
ID de producto                                     1026615 non-null int64
Campaña                                            1026615 non-null object
ID de la campaña                                   1026615 non-null int64
Tipo de producto (primer nivel)                    1026615 non-null object
Tipo de producto (segundo nivel)                   1026615 non-null object
Día                                                1026615 non-null object
Etiqueta personalizada 1                           1026615 non-null object
CPC máximo predeterminado del grupo de anuncios    1026615 non-null object
Moneda                                             1026615 non-null object
Impresiones                                        1026615 non-null int64
Clics                                              1026615 non-null int64
CTR                                                1026615 non-null

Firstly, I rename the columns

In [ ]:
df_adwords.columns

In [ ]:
df_adwords = df_adwords.rename(index=str, 
            columns={"ID de producto":"Reference", "Tipo de producto (primer nivel)": "CatN1",
                     "Tipo de producto (segundo nivel)":"CatN2", "Día":"Date", "Etiqueta personalizada 1": "Cat_Price", 
                     "CPC máximo predeterminado del grupo de anuncios": "CPC_max", "CPC medio": "CPC_medio", 
                     "Impresiones":"Impressions", "Coste":"Cost", "Conversiones": "Conversions",
                    "Todas las conversiones": "All_Conversions", "Valor de conv.": "Ads_Income", 
                    "Valor de todas las conversiones": "Ads_Income_All"})

In [ ]:
Then, I select the columns of our interest for this study

In [ ]:

column_interest_adwords = ["Reference", "CatN1", "CatN2", "Date", "Cat_Price", "CPC_max", "CPC_medio", "Impressions", 
                           "Clics", "Cost", "Conversions", "All_Conversions", "Ads_Income", "Ads_Income_All"]
df_adwords = df_adwords[column_interest_adwords]

#Change reference value from int to string in order to save references id as list later
df_adwords["Reference"] = df_adwords["Reference"].apply(str)

#Parse time to "Date" column
df_adwords["Date"] = pd.to_datetime(df_adwords["Date"], format = "%d/%m/%Y")

#Change to float type
#float_columns = ["CPC_medio", "Cost", "Conversions", "All_Conversions", "Ads_Income", "Ads_Income_All"]
df_adwords["CPC_max"] = df_adwords["CPC_max"].str.replace(",",".").astype(float)
df_adwords["CPC_medio"] = df_adwords["CPC_medio"].str.replace(",",".").astype(float)
df_adwords["Cost"] = df_adwords["Cost"].str.replace(",",".").astype(float)
df_adwords["Conversions"] = df_adwords["Conversions"].str.replace(",",".").astype(float)
df_adwords["All_Conversions"] = df_adwords["All_Conversions"].str.replace(",",".").astype(float)
df_adwords["Ads_Income"] = df_adwords["Ads_Income"].str.replace(",",".").astype(float)
df_adwords["Ads_Income_All"] = df_adwords["Ads_Income_All"].str.replace(",",".").astype(float)

In [ ]:
df_adwords.head(5)

In [ ]:
df_adwords.info()

In [ ]:
df_adwords.shape